In [1]:
import json
import numpy as np
import pandas as pd
import sys, os
#!/usr/bin/python
# parent = os.path.dirname(os.path.realpath(__file__))
sys.path.append('~/mitie')

from mitie import *
from collections import defaultdict

ner = named_entity_extractor('../../../mitie/mitie-v0.2-python-2.7-windows-or-linux64/MITIE-models/english/ner_model.dat')

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from gensim import matutils
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize


/home/ec2-user/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [3]:
from multiprocessing import Pool

In [4]:
from collections import defaultdict
pd.options.display.max_colwidth = 250

In [5]:
vocabdict = defaultdict(int)


### Open and read file

In [6]:
!ls

161102_times.json			 kmeans_test_3.csv
161103_times.json			 kmeans_test_4.csv
161105e_nyt.json			 kmeans_test.csv
161107e_nyt.json			 news
161109_nytm2.json			 nytimes_extended.csv
161109_nytm.json			 nyt nlp final.ipynb
161110_nytm.json			 nyt nlp final-multiday-Copy1.ipynb
161111_nytm.json			 nyt nlp final-multiday.ipynb
161112_nytm.json			 parse_nyt_data-Copy1.ipynb
161114_nytm.json			 parse_nyt_data.ipynb
161116_nytimes_extended_clusters_50.csv  scrapy.cfg
161116_nytimes_extended_clusters.csv	 tfidf_docsimilarity.ipynb
161121nyt2.json				 ward_clusters.png
161121nyt.json				 withspec2.csv
Clusters.png				 withspec.csv
kmeans_test_2.csv


In [7]:
files = ['161102_times.json','161103_times.json','161105e_nyt.json','161105e_nyt.json','161107e_nyt.json','161109_nytm2.json','161110_nytm.json','161114_nytm.json']
splits = []
for file_ in files:
    f = open(file_,'r')
    filestring = f.read()
    split = filestring.replace(']\n','').replace('[\n','').split('\n')
    splits.extend(split)
final_dict = {}

In [8]:
print len(splits)

40472


### Preprocess text

In [132]:
for i in splits:
    try:
        temp = json.loads(i[:-1])
    except:
        continue   
     
    key = temp.keys()[0].split('?')[0]
    if '?' in key:
        print key
        break
    val = temp.values()[0].encode('utf-8').replace("\xe2\x80\x99","'").replace("\xe2\x80\x9c",'"').replace("\xe2\x80\x9d",'"').replace('\xe2\x80\x94',"'").replace('\xe2\x80\x98',"'")
    if val != '':
        final_dict[key] = val
    else:
        continue

In [131]:
final_dict.keys()[:10]

[u'http://www.nytimes.com/2014/01/10/world/middleeast/syrian-groups-try-to-recruit-us-travelers.html',
 u'http://www.nytimes.com/2014/03/19/sports/baseball/a-fast-growing-fraternity-in-the-game-tommy-john-patients.html',
 u'http://www.nytimes.com/2009/07/04/us/politics/04alaska.html',
 u'http://www.nytimes.com/2015/08/01/business/into-the-family-business-at-perdue.html',
 u'http://www.nytimes.com/2010/03/08/world/middleeast/08sunnis.html',
 u'http://www.nytimes.com/2009/06/17/world/middleeast/17tehran.html',
 u'http://www.nytimes.com/2015/04/25/opinion/laurent-fabius-our-climate-imperatives.html',
 u'http://www.nytimes.com/2015/02/17/opinion/yanis-varoufakis-no-time-for-games-in-europe.html',
 u'http://www.nytimes.com/2014/02/11/nyregion/de-blasio-state-of-the-city-speech.html',
 u'http://www.nytimes.com/2015/07/10/us/politics/jeb-bush-races-past-rivals-in-fund-raising-aided-by-super-pac-cash.html']

### Create dataframe

In [34]:
dump_dict = json.dumps(final_dict)
df = pd.read_json(dump_dict, typ = 'series')
df = pd.DataFrame(df).reset_index()
df.columns  = [['url','text']]
df['date'] = df['url'].apply(lambda x: x.split('.com/')[1].split('/')[:3])
df['section'] = df['url'].apply(lambda x: x.split('.com/')[1].split('/')[3])
df['length'] = df['text'].apply(lambda x: len(x))

In [35]:
df['length'].sum()

155974508

### Named Entitiy Recognition

In [13]:
import time

In [14]:
t0 = time.time()

In [15]:
cv2 = CountVectorizer(stop_words='english',max_df = 0.15, min_df = 0.01, strip_accents = 'unicode',analyzer='word').fit(df['text'])

In [16]:
t1 = time.time()

In [17]:
print t1-t0

45.7455041409


# THIS IS WHERE YOU LEFT OFF DUMMY

In [53]:
## Data should be pre-tokenized within a dataframe, this is intended to be applied to that column of the dataframe and will output  
entities_set = set()


def get_ent_simp(x, entities_set = entities_set):
        entities = ner.extract_entities(x)
        temp_ent = set()
        for e in entities:
            rangee = e[0]
            entity_text = " ".join(x[i] for i in rangee)
            temp_ent.update([entity_text])
        
        entities_set.update(set(temp_ent))
#         print len(set(entities_list))



In [28]:
df['text'][0][26:31]

u'hop F'

In [47]:
final_set = set()

In [48]:

test_set = set()
test_ent = ner.extract_entities(df['tokens'][0])
for e in test_ent:
    rangee = e[0]
    print rangee
    entity_text = " ".join(df['tokens'][0][i] for i in rangee)
    print entity_text
    test_set.update(entity_text)
final_set.update(test_set)

xrange(2)
HONG KONG
xrange(7, 11)
Bishop Francis X. Ford
xrange(12, 13)
Brooklyn-born
xrange(15, 16)
Meihsien
xrange(18, 20)
Kwangtung Province
xrange(27, 28)
Canton
xrange(36, 37)
Chinese
xrange(37, 38)
Communist


In [ ]:
vocabdict= {i:0 for i in cv2.vocabulary_.keys()}

In [ ]:
cv2.vocabulary_.keys()

In [ ]:
def get_vocab(x):
    vocabdict = defaultdict(int)
    for i in x:
        try:
            j = list(set([i]).intersection(set(cv2.vocabulary_.keys())))
            vocabdict[j[0]]+=1
        except:
            continue
            
    returnlist = []
    for k, v in vocabdict.iteritems():
        returnlist.append((k+' ')*v)
    return ' '.join(returnlist)

In [36]:
df['tokens'] = df['text'].apply(lambda x: tokenize(x))

In [54]:
t0 = time.time()
df['tokens'].apply(lambda x: get_ent_simp(x))
t1 = time.time()

In [55]:
t1-t0

447.7204670906067

In [65]:
entities_list = list(entities_set)

In [66]:
entities_list.extend(cv2.vocabulary_.keys())

In [70]:
len(entities_set)

207401

In [71]:
len(cv2.vocabulary_.keys())

5711

In [140]:
entities_list = list(set(entities_list))
len(entities_list)

212996

In [141]:
final_vocab = [i.decode("utf-8").encode("ascii","ignore") for i in entities_list]

In [145]:
final_vocab = list(set(final_vocab))

In [ ]:
t0 = time.time()

In [ ]:
df['vocab'] = df['tokens'].apply(lambda x: get_vocab(x))

In [ ]:
t1 = time.time()
print t1-t0

In [ ]:
print t1-t0
print '29246.0247421'

In [ ]:
df['vocab'][0] + df['entities'][0]

In [ ]:
from datetime import datetime
print datetime.fromtimestamp(t0)
print datetime.fromtimestamp(t1)

In [ ]:
def get_entities(x, cw=cv2.vocabulary_.keys()):
    x = tokenize(x)
    entities = ner.extract_entities(x)
    entities_list = []
    for e in entities:
        rangee = e[0]
        entity_text = " ".join(x[i] for i in rangee)
        entities_list.append(entity_text)
    for i in x:
        if i.lower() in cw:
            entities_list.append(i)
    return entities_list
        

#### Lemmatization for Count Vectorizer

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemmatize_me(x):
    x = word_tokenize(x)
    x_tagged = nltk.pos_tag(x)
    return_list = []
    for i in x_tagged:
        if 'V' in i[1]:
            value = 'v'
        else:
            value = 'n'
        return_list.append(lemmatizer.lemmatize(i[0], pos = value))
    return return_list


#### create countvectorizer object, transform text to sparse matrix using named entitiy processor

In [ ]:
import gc
gc.collect()

In [ ]:
t0 = time.time()

In [ ]:
cv = CountVectorizer(stop_words='english', max_df = 0.15, strip_accents='unicode').fit(df['vocab'])

In [ ]:
t1 = time.time()

In [ ]:
t1-t0

In [ ]:
t0 = time.time()

In [ ]:
sm = cv.transform(df['vocab'])

In [ ]:
t1 = time.time()

In [ ]:
t1-t0

### Fit TF IDF and transform sparse matrix from count vectorizer

In [ ]:
t0 = time.time()

In [ ]:
tt = TfidfTransformer(sm)

In [ ]:
t1 = time.time()

In [ ]:
t1-t0

In [ ]:
tt

In [158]:
t0 = time.time()

In [164]:
tv = TfidfVectorizer( stop_words='english',vocabulary=final_vocab).fit(df['text'])

In [165]:
tfidf = tv.transform(df['text'])

In [166]:
t1 = time.time()

In [167]:
t1-t0

192.42994904518127

In [168]:
tfidf

<25759x212275 sparse matrix of type '<type 'numpy.float64'>'
	with 5245638 stored elements in Compressed Sparse Row format>

In [202]:
import pickle
pickle.dump(final_vocab,open('nytimes_test_vocab.pkl','wb'))

In [205]:
final_vocab

['',
 'Mark Sisson',
 'Mr Cortez',
 'Wexner Center',
 'Barry Berke',
 'Mr Cortes',
 'Fulton Fish Market',
 'Safarova',
 'Nampo',
 'Mr Van Haeften',
 'Paul McGill',
 'Yousef Zemmouri',
 'Elise Landgraf',
 'Janice Sage',
 'Dr Vianna',
 'Archuleta',
 'Lincoln Town Car',
 'Gravesend',
 'Howyis',
 "Xi 'an Environmental Protection Bureau",
 'S-5',
 'Euro',
 'Loc Tran',
 'Valli',
 'Andrew D. Hamingson',
 'Bill Mazer',
 'Valle',
 "Luhansk People 's Republics",
 'Willow Song',
 'Nicole Walden',
 'Picture ,',
 'Johns Hopkins Nitze School of Advanced International Studies',
 'PORTLAND',
 'Mr Jimenez',
 'DeGrom',
 'Ms Brownstein',
 'Valls',
 'receiving',
 'Saca',
 'Mike Holloway',
 'Housing and Urban Development',
 'R.V.s',
 'Miers',
 'Leonard Fournette',
 'Sergeants',
 'Sack',
 'Caesar Cella',
 'John Battelle',
 'Tomas Plekanec',
 'Pinkerton',
 'Cagelle',
 'Qin Guangrong',
 'Mr Lambert',
 "Urban Institute 's Health Policy Center",
 'Tommy Moore',
 'Daw Kyi Kyi Tun',
 'Bassanio',
 'Daphni Leef',
 

#### Latent Semantic Analysis

##### Trained on count vectorizer sparse matrix, better explained variance ratio

In [ ]:
lsa = TruncatedSVD(n_components=20)
nyt_lsa = lsa.fit_transform(sm)
nyt_lsa = Normalizer(copy=False).fit_transform(nyt_lsa)

In [ ]:
nyt_lsa

In [ ]:
lsa.explained_variance_

#### Let's do the damn thing. Kmeans 

In [ ]:
kmeans = KMeans(n_clusters=7)
km = kmeans.fit(tt)
clusters = km.labels_.tolist()

In [ ]:
df['clusters'] = clusters
df['clusters2'] = clusters2

In [ ]:
df[['url','clusters','clusters2','spec']].to_csv('withspec2.csv')

In [ ]:
kmeans2 = KMeans(n_clusters=50)
km2 = kmeans2.fit(tfidf)
clusters2 = km2.labels_.tolist()

#### Spectral wtf is this shit?

In [169]:
import gc
gc.collect()

0

In [170]:
from sklearn.cluster import SpectralClustering

In [181]:
spec = SpectralClustering(n_clusters=100, affinity='cosine', n_init=50)

In [182]:
spec_fit = spec.fit(tfidf)
spec_pred = spec.fit_predict(tfidf)
df['spec'] = spec_pred

In [183]:
df_write = df.drop(['text'], axis=1)

In [184]:
# df_write.to_csv('161116_nytimes_extended_clusters_50.csv')

In [185]:
af_mat = pd.DataFrame(spec_fit.affinity_matrix_)

In [186]:
df.groupby('spec')['url'].count().mean()- 0.5*df.groupby('spec')['url'].count().std()


83.60146422592064

In [195]:
print df.groupby('spec')['url'].count() 

spec
0      253
1      392
2       65
3      203
4      592
5      210
6      322
7       54
8      176
9      212
10     476
11     196
12     457
13     395
14      90
15      94
16     872
17     269
18     206
19      98
20     386
21     177
22     162
23     158
24     260
25     185
26     326
27     201
28      90
29     209
30     132
31     155
32    3390
33      93
34      97
35     106
36     152
37     114
38     165
39     252
40     285
41     276
42      72
43     500
44     250
45     248
46     268
47     102
48      86
49      97
50     539
51     281
52     346
53     189
54     234
55     164
56      55
57     190
58     177
59     249
60     124
61     188
62     364
63     261
64     153
65     168
66     107
67     160
68     298
69     449
70     205
71     154
72     149
73     259
74     171
75     162
76     173
77      69
78     368
79     168
80     284
81     315
82     171
83     179
84     789
85     191
86     115
87      91
88     103
89     412
90   

#### af_mat[5].sort(inplace=False, ascending=False)[:10].mean()

In [194]:
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 250

In [196]:
print df[['url']][df['spec'] ==42]

                                                                                                                                             url
130                                                         http://www.nytimes.com/1999/08/07/sports/baseball-another-day-another-milestone.html
8777                                 http://www.nytimes.com/2013/07/13/sports/baseball/daviss-35th-homer-helps-orioles-power-past-blue-jays.html
11164                                  http://www.nytimes.com/2014/03/24/sports/baseball/dodgers-beat-diamondbacks-7-5-for-down-under-sweep.html
11520                                                                    http://www.nytimes.com/2014/04/23/sports/baseball/baseball-roundup.html
11570                                                                         http://www.nytimes.com/2014/04/26/sports/baseball/mlb-roundup.html
11751                                                                         http://www.nytimes.com/2014/05/13/sports/baseball/ml

In [ ]:
# spec = SpectralClustering(n_clusters=50, eigen_solver='arpack', affinity='sigmoid') #good clustering for certain topics mosul(7??) comey, clinton, trump, india 12, 

In [ ]:
spec_fit.n_neighbors

In [ ]:
"""
Visualization of matrix block structure
(eg, pairwise similarity or co-occurrence)
Requires scaledimage.py for intensity plots
David Andrzejewski
"""
import numpy as NP
import matplotlib.pyplot as P
import matplotlib.lines as L
import sklearn.cluster as SKLC

# from scaledimage import scaledimage

def blockviz(affinity, nclust, ax=None):
    """ 
    Visualize block-structure of affinity matrix 
    affinity = NxN non-negative affinity matrix
    nclust = number of clusters to use
    ax = matplotlib Axes to draw on 
    Rely on caller to .show()
    """
    # Activate the appropriate Axes in pyplot
    if(ax == None):
        ax = P.figure().gca()
    # Do spectral clustering
    ndata = affinity.shape[0]
    c = SKLC.SpectralClustering(n_clusters=nclust, affinity='linear', random_state=1445)
    c.fit(affinity)
    # Extract cluster labels and sort indices to align with clusters
    sortidx = []
    for ki in range(nclust):
        sortidx += getlabeled(c.labels_, ki)
    sorted_affinity = affinity.copy()
    sorted_affinity = sorted_affinity[sortidx,:]
    sorted_affinity = sorted_affinity[:,sortidx]
    # Intensity plot of affinity
    width = 10
    scaledimage(sorted_affinity,
                pixwidth=width, grayscale=True, ax=ax)
    # Draw recovered cluster boundaries
    kstart = 0
    for ki in range(nclust):    
        clustki = getlabeled(c.labels_, ki)
        drawClust(ax, 
                  kstart,
                  kstart + len(clustki),
                  ndata,
                  scale=width)
        kstart += len(clustki)
    return c.labels_

def logistic(val):
    """ Logistic function """
    return float(1) / (1 + NP.exp(-1 * val))

def getlabeled(labels, ki):
    """ Get indices where labels==ki """
    return [idx for (idx, val) in 
            enumerate(labels) if val == ki]

def drawH(ax, y, xstart, xend):
    """ Draw horiztonal line """
    ax.add_line(L.Line2D([xstart, xend], 
                  [y, y],
                  color='r'))

def drawV(ax, x, ystart, yend):
    """ Draw vertical line """
    ax.add_line(L.Line2D([x, x],
                         [ystart, yend], 
                         color='r'))

def drawClust(ax, kstart, kend, kmax, scale=1):
    """ Draw bounding box for cluster """
    skstart = scale * kstart
    skend = scale * kend
    skmax = scale * kmax
    if(skstart == 0):
        # Upper-left cluster: only draw bottom-right borders
        drawH(ax, skmax-skend, skstart, skend)
        drawV(ax, skend, skmax-skstart, skmax-skend)
    elif(skend == skmax):
        # Lower-right cluster: only draw top-left borders
        drawH(ax, skmax-skstart, skstart, skend)
        drawV(ax, skstart, skmax-skstart, skmax-skend)
    else:
        # Otherwise, draw all 4 borders
        drawH(ax, skmax-skend, skstart, skend)
        drawV(ax, skend, skmax-skstart, skmax-skend)
        drawH(ax, skmax-skstart, skstart, skend)
        drawV(ax, skstart, skmax-skstart, skmax-skend)
        
import numpy as NP
import matplotlib.pyplot as P
import matplotlib.ticker as MT
import matplotlib.cm as CM
 
def scaledimage(W, pixwidth=20, ax=None, grayscale=True):
    """
    Do intensity plot, similar to MATLAB imagesc()
 
    W = intensity matrix to visualize
    pixwidth = size of each W element
    ax = matplotlib Axes to draw on 
    grayscale = use grayscale color map
 
    Rely on caller to .show()
    """
    # N = rows, M = column
    (N, M) = W.shape 
    # Need to create a new Axes?
    if(ax == None):
        ax = P.figure(figsize=(20,20)).gca()
    # extents = Left Right Bottom Top
    exts = (0, pixwidth * M, 0, pixwidth * N)
    if(grayscale):
        ax.imshow(W,
                  interpolation='nearest',
                  cmap=CM.gray,
                  extent=exts)
    else:
        ax.imshow(W,
                  interpolation='nearest',
                  extent=exts)
 
    ax.xaxis.set_major_locator(MT.NullLocator())
    ax.yaxis.set_major_locator(MT.NullLocator())
    return ax
 
if __name__ == '__main__':
    # Define a synthetic test dataset
     testweights = NP.array([[0.25, 0.50, 0.25, 0.00],
                            [0.00, 0.50, 0.00, 0.00],
                            [0.00, 0.10, 0.10, 0.00],
                            [0.00, 0.00, 0.25, 0.75]])
    # Display it
#     ax = scaledimage(testweights)
#     P.show()


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

####  cosine, 25 clusters

In [ ]:
blockviz(np.matrix(af_mat), 25, ax=P.figure(figsize=(10,10)).gca())

#### cosine 30^

In [ ]:
blockviz(np.matrix(af_mat), 20)

#### cosine 20 ^

In [ ]:
blockviz(np.matrix(af_mat), 25, ax=P.figure(figsize=(100,100)).gca())


#### cosine 25

In [ ]:
blockviz(np.matrix(af_mat), 20)

#### poly 50 ^

In [ ]:
blockviz(np.matrix(af_mat), 20)

#### poly 30

In [ ]:
blockviz(np.matrix(af_mat), 20)

#### Poly 20

In [ ]:
import seaborn as sns

In [ ]:
df

In [ ]:
from multiprocessing import Pool

def tfidf(x):
    tfidf = TfidfVectorizer(analyzer= get_entities, stop_words='english', max_df=0.15, strip_accents='unicode').fit_transform(x)
    return tfidf

if __name__ == '__main__':
    p = Pool(16)
    print(p.map(tfidf, df['text'] ))

In [ ]:
6+7

In [ ]:
x = ['hi','there','my','name','is','what','the','actual','fuck']
cw = ['hi','fuck','how','the','fuck','are','you']

In [ ]:
" ".join(get_vocab(x, cw))

In [ ]:
' '.join(x)

In [ ]:
tv2 = TfidfVectorizer(stop_words='english', max_df=0.15, strip_accents='unicode').fit(df['final_str'])


In [ ]:
tfidf2 = tv2.transform(df['final_str'])

In [ ]:
spec2 = SpectralClustering(n_clusters=20, affinity='cosine', n_init=50)
spec2_fit = spec.fit(tfidf2)
spec2_pred = spec.fit_predict(tfidf2)
df['spec2'] = spec2_pred

In [ ]:
pd.options.display.max_colwidth = 250


In [ ]:
print df[['url']][df['spec2'] ==28]

In [ ]:
df['final'] = df['tokens'].apply(lambda x: get_vocab(x))

In [ ]:
outlist = []
for k, v in voc_dict.iteritems():
        outlist.append((k+' ')*v)

In [ ]:
' '.join(outlist)

In [ ]:
list(set(df['tokens'][10]).intersection(set(cv2.vocabulary_.keys())))

In [ ]:
len(cv2.vocabulary_)

In [ ]:
set

In [ ]:
vocabdict = defaultdict(int)


In [ ]:
vocabdict['fuck'] +=1

In [ ]:
vocabdict['fuck']